In [1]:
import onnx 
import torch
from torch.nn import Linear
import torch.nn.functional as F
from brevitas.nn import QuantLinear, QuantReLU, QuantIdentity

In [2]:
import numpy as np
import random
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_seed(42)

In [3]:
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import FacebookPagePage
from torch_geometric.utils import degree, to_dense_adj
from torch_geometric.transforms import GCNNorm
from collections import Counter
import matplotlib.pyplot as plt

In [4]:
dataset = Planetoid(root='.', name = 'Cora')
data = dataset[0]

In [5]:
# Import dataset from PyTorch Geometric
dataset = FacebookPagePage(root=".")

data = dataset[0]

# Print information about the dataset
print(f'Dataset: {dataset}')
print('-----------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

# Create masks
data.train_mask = range(18000)
data.val_mask = range(18001, 20000)
data.test_mask = range(20001, 22470)

Dataset: FacebookPagePage()
-----------------------
Number of graphs: 1
Number of nodes: 22470
Number of features: 128
Number of classes: 4

Graph:
------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: True


In [6]:
degrees = degree(data.edge_index[0]).numpy()
numbers = Counter(degrees)

## Now we define our GCN

In [11]:
def accuracy(y_pred, y_true):
    """Calculate accuracy."""
    return torch.sum(y_pred == y_true) / len(y_true)

In [12]:
from torch_geometric.nn.conv.gcn_conv import gcn_norm
norm_edge_index, norm_edge_weight = gcn_norm(
    edge_index=data.edge_index,
    edge_weight=None, # Assuming unweighted graph initially
    num_nodes=data.num_nodes,
    improved=False,
    add_self_loops=True,
    dtype=torch.float32
)

In [13]:
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_sparse import SparseTensor

# This uses norm_edge_index and norm_edge_weight from your gcn_norm() call
sparse_adj = SparseTensor.from_edge_index(
    norm_edge_index,
    norm_edge_weight,
    sparse_sizes=(data.num_nodes, data.num_nodes)
)

In [19]:
class QuantGCNLayer(torch.nn.Module):
    def __init__(self, in_features, out_features, weight_bit_width=8, act_bit_width=8, bias=True, bias_quantizer=False, return_quant_tensor=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
    # Quantized linear layer for X * W
        self.linear = QuantLinear(
            in_features=in_features,
            out_features=out_features,
            bias=bias,
            weight_bit_width=weight_bit_width,
            bias_quantizer=bias_quantizer, # Pass bias quantizer if you define one
            return_quant_tensor=return_quant_tensor
        )

    def forward(self, x, sparse_norm_adj):
        # 1. Linear transformation (X * W_quant)
        # If x is float, QuantLinear quantizes it first internally based on its input_quantizer.
        # If x is QuantTensor, QuantLinear uses its scale.
        transformed_features = self.linear(x) # Output is a QuantTensor if return_quant_tensor=True

        # 2. Graph propagation ((D^-1/2 * A_hat * D^-1/2) * XW)
        # torch.mm can handle if one input is QuantTensor and other is float.
        # The output will be a QuantTensor preserving the scale from transformed_features.
        output_features = sparse_norm_adj.matmul(transformed_features.tensor)

        return output_features

    def __repr__(self):
        return f'{self.__class__.__name__}({self.in_features} -> {self.out_features})'       

In [20]:
from brevitas.quant.scaled_int import Int8Bias
class QuantGCN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out,
                 weight_bit_width=8, act_bit_width=8, in_bit_width=8,
                 bias_quantizer=Int8Bias,
                 mode = 'train'): # Define if you want specific bias quantization
        super().__init__()
        self.mode = mode
        # Input quantizer for the raw node features 'x'
        self.input_quant = QuantIdentity(
            bit_width=in_bit_width,
            return_quant_tensor=True
        )

        # First GCN layer
        self.conv1 = QuantGCNLayer(
            dim_in, dim_h,
            weight_bit_width=weight_bit_width,
            bias=True, # Standard GCNConv has bias
            bias_quantizer=bias_quantizer,
            return_quant_tensor=True
        )

        # Quantized ReLU activation
        self.relu1 = QuantReLU(
            bit_width=act_bit_width,
            return_quant_tensor=True
        )

                # First GCN layer
        self.conv2 = QuantGCNLayer(
            dim_h, dim_out,
            weight_bit_width=weight_bit_width,
            bias=True, # Standard GCNConv has bias
            bias_quantizer=bias_quantizer,
            return_quant_tensor=True
        )

        self.output_dequant = QuantIdentity(
            return_quant_tensor=False # Returns a float torch.Tensor
        )
        self.output_quant = QuantIdentity(
        return_quant_tensor=True # Returns a QuantReLU tensor
        )


    def forward(self, x, dense_norm_adj):
        """
        x: Raw node features, shape (num_nodes, dim_in)
        dense_norm_adj: Pre-computed dense normalized adjacency matrix,
                        shape (num_nodes, num_nodes)
        """
        # 1. Quantize input features
        x_quant = self.input_quant(x)

        # 2. First GCN layer + ReLU
        h1 = self.conv1(x_quant, dense_norm_adj)
        h1_activated = self.relu1(h1)

        # 3. Second GCN layer
        h2 = self.conv2(h1_activated, dense_norm_adj)
        
        # 4. Dequantize output for training loss calculation
        mode = self.mode
        final_logits = self.output_dequant(h2)
        export_logits = self.output_quant(h2)
        # For comparison with a PyG GCN model that uses F.log_softmax:
        if (mode=='train'):
            return F.log_softmax(final_logits, dim=1)
        # For training with CrossEntropyLoss and for FINN (which prefers logits):
        else:
            
            return export_logits
            #return final_logits

    def fit(self, data, dense_norm_adj, epochs, lr=0.001, weight_decay=5e-4):
        # Ensure data.y is prepared (e.g., torch.long, correct device)
        # Ensure dense_norm_adj is on the correct device
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=weight_decay)

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()

            # Ensure features and adjacency are on the same device as the model
            features = data.x.to(next(self.parameters()).device)
            adj = dense_norm_adj.to(next(self.parameters()).device)
            
            out = self(features, adj)
            
            train_mask_device = data.train_mask
            labels_device = data.y

            loss = criterion(out[train_mask_device], labels_device[train_mask_device])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                                  data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')

    @torch.no_grad()
    def test(self, data, dense_norm_adj):
        self.eval()
        mode = self.mode
        print(mode)
        features = data.x.to(next(self.parameters()).device)
        adj = dense_norm_adj.to(next(self.parameters()).device)
        out = self(features, adj)
        if (mode =='train'):
            out = out
        else:
            out = out.tensor
        test_mask_device = data.test_mask
        labels_device = data.y
        acc = accuracy(out.argmax(dim=1)[test_mask_device], labels_device[test_mask_device])
        return acc

In [21]:
w = 8
a = 8
wa = 'w8a8'

In [22]:
quant_gcn = QuantGCN(dataset.num_features, 64, dataset.num_classes, weight_bit_width=w, act_bit_width=a, in_bit_width=32, mode='train')
print(quant_gcn)

QuantGCN(
  (input_quant): QuantIdentity(
    (input_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (act_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
      (fused_activation_quant_proxy): FusedActivationQuantProxy(
        (activation_impl): Identity()
        (tensor_quant): RescalingIntQuant(
          (int_quant): IntQuant(
            (float_to_int_impl): RoundSte()
            (tensor_clamp_impl): TensorClamp()
            (delay_wrapper): DelayWrapper(
              (delay_impl): _NoDelay()
            )
          )
          (scaling_impl): ParameterFromRuntimeStatsScaling(
            (stats_input_view_shape_impl): OverTensorView()
            (stats): _Stats(
              (stats_impl): AbsPercentile()
            )
            (restrict_scaling): _RestrictValue(
              (restrict_value_impl): FloatRestrictValue()
            )
            (clamp_scaling): _ClampValue(
              (clamp_min_

In [23]:
# Train
quant_gcn.fit(data,sparse_adj, epochs=260, lr = 0.005)

Epoch   0 | Train Loss: 1.392 | Train Acc: 21.88% | Val Loss: 1.39 | Val Acc: 22.86%
Epoch  20 | Train Loss: 0.488 | Train Acc: 83.22% | Val Loss: 0.49 | Val Acc: 83.64%
Epoch  40 | Train Loss: 0.322 | Train Acc: 89.74% | Val Loss: 0.31 | Val Acc: 89.74%
Epoch  60 | Train Loss: 0.274 | Train Acc: 91.44% | Val Loss: 0.27 | Val Acc: 91.70%
Epoch  80 | Train Loss: 0.251 | Train Acc: 92.47% | Val Loss: 0.25 | Val Acc: 92.70%
Epoch 100 | Train Loss: 0.235 | Train Acc: 93.10% | Val Loss: 0.24 | Val Acc: 93.15%
Epoch 120 | Train Loss: 0.223 | Train Acc: 93.50% | Val Loss: 0.23 | Val Acc: 93.45%
Epoch 140 | Train Loss: 0.214 | Train Acc: 93.83% | Val Loss: 0.22 | Val Acc: 93.50%
Epoch 160 | Train Loss: 0.204 | Train Acc: 94.14% | Val Loss: 0.22 | Val Acc: 93.55%
Epoch 180 | Train Loss: 0.197 | Train Acc: 94.33% | Val Loss: 0.21 | Val Acc: 93.60%
Epoch 200 | Train Loss: 0.190 | Train Acc: 94.60% | Val Loss: 0.21 | Val Acc: 93.70%
Epoch 220 | Train Loss: 0.184 | Train Acc: 94.78% | Val Loss: 0.2

In [25]:
# Test
acc = quant_gcn.test(data, sparse_adj)
print(f'\nGCN test accuracy: {acc*100:.2f}%\n')

train

GCN test accuracy: 93.60%



In [26]:
export_path = f'quant_gcn_trained_weights_{wa}.pth'
torch.save(quant_gcn.state_dict(), export_path)

In [27]:
quant_gcn_export = QuantGCN(dataset.num_features, 64, dataset.num_classes, weight_bit_width=w, act_bit_width=a, in_bit_width=32, mode='export')

In [28]:
trained_weights = torch.load(export_path)

In [29]:
quant_gcn_export.load_state_dict(trained_weights)

<All keys matched successfully>

In [31]:
acc2 = quant_gcn_export.test(data, sparse_adj)
print(f'\nGCN test accuracy for {wa}: {acc2*100:.2f}%\n')

export

GCN test accuracy for w8a8: 93.60%



## now to export

In [38]:
feat_np = data.x.numpy().astype(np.float32)
norm_edge_index, norm_edge_weight = gcn_norm(
    edge_index=data.edge_index,
    edge_weight=None,
    num_nodes=data.num_nodes,
    improved=False,
    add_self_loops=True,
    dtype=torch.float32
)
edge_index_np = norm_edge_index.numpy().astype(np.int64)
edge_weight_np = norm_edge_weight.numpy().astype(np.float32)

feat_np = np.ascontiguousarray(feat_np)
edge_index_np = np.ascontiguousarray(edge_index_np)
edge_weight_np = np.ascontiguousarray(edge_weight_np)


In [39]:
np.save('features.npy', feat_np)
np.save('edge_index.npy', edge_index_np)
np.save('edge_weight.npy', edge_weight_np)

In [40]:
import torch.nn as nn

In [41]:
class CleanGCNLayer(torch.nn.Module):
    def __init__(self, in_feats, out_feats, bias=True):
        super().__init__()
        self.linear = nn.Linear(in_feats, out_feats, bias=bias)

    def forward(self, x, adj):
        return adj @ self.linear(x)

class CleanGCN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.conv1 = CleanGCNLayer(dim_in, dim_h)
        self.relu1 = nn.ReLU()
        self.conv2 = CleanGCNLayer(dim_h, dim_out)

    def forward(self, x, adj):
        h = self.relu1(self.conv1(x, adj))
        return self.conv2(h, adj)


In [42]:
brevitas_weights = torch.load(f'quant_gcn_trained_weights_{wa}.pth', map_location='cpu')

# Mapping to clean model
mapped_weights = {
    'conv1.linear.weight': brevitas_weights['conv1.linear.weight'],
    'conv1.linear.bias':   brevitas_weights['conv1.linear.bias'],
    'conv2.linear.weight': brevitas_weights['conv2.linear.weight'],
    'conv2.linear.bias':   brevitas_weights['conv2.linear.bias'],
}

clean_model = CleanGCN(dim_in=dataset.num_features, dim_h=64, dim_out=dataset.num_classes)
clean_model.load_state_dict(mapped_weights)
clean_model.eval()


CleanGCN(
  (conv1): CleanGCNLayer(
    (linear): Linear(in_features=128, out_features=64, bias=True)
  )
  (relu1): ReLU()
  (conv2): CleanGCNLayer(
    (linear): Linear(in_features=64, out_features=4, bias=True)
  )
)

In [43]:
dummy_x = torch.randn(data.num_nodes, dataset.num_features, dtype=torch.float32)
dummy_adj = torch.randn(data.num_nodes, data.num_nodes, dtype=torch.float32)

torch.onnx.export(
    clean_model,
    (dummy_x, dummy_adj),
    f"clean_gcn_export_{wa}.onnx",
    input_names=['global_in', 'global_in_1'],
    output_names=['global_out'],
    opset_version=13
)
print(f'Exported to clean_gcn_export_{wa}.onnx')

Exported to clean_gcn_export_w8a8.onnx


# now move to the fpga and get the data

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from torch_geometric.datasets import Planetoid

# Load your FPGA output (logits)
fpga_output = np.load(f'fpga_output_{wa}.npy')  # shape: (2708, num_classes)

# Convert logits to predictions (class with highest score)
preds = np.argmax(fpga_output, axis=1)

# Load true labels from the dataset
dataset = Planetoid(root="data/Cora", name="Cora")
true_labels = dataset[0].y.numpy()

# Calculate accuracy
accuracy = accuracy_score(true_labels, preds)
print(f"Accuracy on FPGA output for {wa}: {accuracy * 100:.2f}%")

# Detailed classification report
report = classification_report(true_labels, preds, digits=4)
print(f"Classification Report for {wa}:\n", report)


## Now we generate graphs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report
import pandas as pd

# Set style for publication-quality plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Your experimental data
quantization_levels = ['W32A32', 'W16A16', 'W8A8', 'W4A4', 'W2A2']
bit_widths = [32, 16, 8, 4, 2]

# Training accuracies (from your results)
training_accuracy = [80.9, 80.90, 81.00, 80.90, 75.10]

# FPGA accuracies (from your results)
fpga_accuracy = [80.9, 79.95, 79.87, 79.95, 74.85]  # Assuming W32A32 same as training

# Runtime data (seconds)
runtime = [2.5, 2.5408, 2.4667, 2.0962, 2.4624]  # Estimated W32A32

# Throughput (nodes/sec)
throughput = [1065, 1065.79, 1097.82, 1291.87, 1099.72]  # Estimated W32A32

# Power estimation (ARM core 400mW)
power_consumption = 0.4  # Watts
energy_consumption = [power_consumption * t for t in runtime]  # Joules

# Classification metrics (from your classification reports)
# You'll need to extract these from your sklearn classification reports
precision_scores = {
    'W32A32': [0.68, 0.76, 0.86, 0.89, 0.76, 0.85, 0.66],  # Estimated based on pattern
    'W16A16': [0.6831, 0.7550, 0.8638, 0.8882, 0.7631, 0.8500, 0.6623],
    'W8A8': [0.6849, 0.7550, 0.8638, 0.8879, 0.7615, 0.8500, 0.6565],
    'W4A4': [0.6510, 0.7602, 0.8509, 0.8841, 0.8041, 0.8340, 0.6901],
    'W2A2': [0.6842, 0.7830, 0.5787, 0.9257, 0.7817, 0.9202, 0.7026]
}

recall_scores = {
    'W32A32': [0.71, 0.86, 0.93, 0.74, 0.85, 0.74, 0.84],  # Estimated
    'W16A16': [0.7123, 0.8664, 0.9258, 0.7384, 0.8545, 0.7416, 0.8389],
    'W8A8': [0.7123, 0.8664, 0.9258, 0.7359, 0.8545, 0.7416, 0.8389],
    'W4A4': [0.7493, 0.8618, 0.9282, 0.7457, 0.8192, 0.7416, 0.8167],
    'W2A2': [0.7407, 0.7650, 0.9856, 0.6088, 0.8404, 0.6577, 0.7611]
}

f1_scores = {
    'W32A32': [0.69, 0.81, 0.89, 0.81, 0.80, 0.79, 0.74],  # Estimated
    'W16A16': [0.6974, 0.8069, 0.8938, 0.8064, 0.8062, 0.7921, 0.7402],
    'W8A8': [0.6983, 0.8069, 0.8938, 0.8048, 0.8053, 0.7921, 0.7366],
    'W4A4': [0.6967, 0.8078, 0.8879, 0.8090, 0.8116, 0.7851, 0.7481],
    'W2A2': [0.7114, 0.7739, 0.7292, 0.7345, 0.8100, 0.7671, 0.7307]
}

# Calculate macro averages
macro_precision = [np.mean(precision_scores[level]) for level in quantization_levels]
macro_recall = [np.mean(recall_scores[level]) for level in quantization_levels]
macro_f1 = [np.mean(f1_scores[level]) for level in quantization_levels]

def create_accuracy_comparison_plot():
    """Plot training vs FPGA accuracy comparison"""
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    x = np.arange(len(quantization_levels))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, training_accuracy, width, label='Training Accuracy', alpha=0.8)
    bars2 = ax.bar(x + width/2, fpga_accuracy, width, label='FPGA Accuracy', alpha=0.8)
    
    ax.set_xlabel('Quantization Level', fontsize=12)
    ax.set_ylabel('Accuracy (%)', fontsize=12)
    ax.set_title('GCN Accuracy: Training vs FPGA Deployment', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(quantization_levels)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(70, 85)
    
    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.1f}%',
                       xy=(bar.get_x() + bar.get_width() / 2, height),
                       xytext=(0, 3),
                       textcoords="offset points",
                       ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    return fig

def create_performance_metrics_plot():
    """Plot performance metrics (throughput, energy, runtime)"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Throughput plot
    bars1 = ax1.bar(quantization_levels, throughput, color='skyblue', alpha=0.8)
    ax1.set_ylabel('Throughput (nodes/sec)', fontsize=12)
    ax1.set_title('Throughput vs Quantization Level', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    for bar, val in zip(bars1, throughput):
        ax1.annotate(f'{val:.0f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    
    # Energy consumption plot
    bars2 = ax2.bar(quantization_levels, energy_consumption, color='lightcoral', alpha=0.8)
    ax2.set_ylabel('Energy Consumption (Joules)', fontsize=12)
    ax2.set_title('Energy Consumption vs Quantization Level', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    for bar, val in zip(bars2, energy_consumption):
        ax2.annotate(f'{val:.2f}J', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    
    # Runtime plot
    bars3 = ax3.bar(quantization_levels, runtime, color='lightgreen', alpha=0.8)
    ax3.set_ylabel('Runtime (seconds)', fontsize=12)
    ax3.set_title('Runtime vs Quantization Level', fontsize=12, fontweight='bold')
    ax3.set_xlabel('Quantization Level', fontsize=12)
    ax3.grid(True, alpha=0.3)
    for bar, val in zip(bars3, runtime):
        ax3.annotate(f'{val:.2f}s', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    
    # Energy efficiency (throughput/energy)
    energy_efficiency = [t/e for t, e in zip(throughput, energy_consumption)]
    bars4 = ax4.bar(quantization_levels, energy_efficiency, color='gold', alpha=0.8)
    ax4.set_ylabel('Energy Efficiency (nodes/Joule)', fontsize=12)
    ax4.set_title('Energy Efficiency vs Quantization Level', fontsize=12, fontweight='bold')
    ax4.set_xlabel('Quantization Level', fontsize=12)
    ax4.grid(True, alpha=0.3)
    for bar, val in zip(bars4, energy_efficiency):
        ax4.annotate(f'{val:.0f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    
    plt.tight_layout()
    return fig

def create_classification_metrics_plot():
    """Plot precision, recall, F1-score"""
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    x = np.arange(len(quantization_levels))
    width = 0.25
    
    bars1 = ax.bar(x - width, macro_precision, width, label='Precision', alpha=0.8)
    bars2 = ax.bar(x, macro_recall, width, label='Recall', alpha=0.8)
    bars3 = ax.bar(x + width, macro_f1, width, label='F1-Score', alpha=0.8)
    
    ax.set_xlabel('Quantization Level', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('Classification Metrics vs Quantization Level', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(quantization_levels)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0.65, 0.85)
    
    # Add value labels
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.3f}',
                       xy=(bar.get_x() + bar.get_width() / 2, height),
                       xytext=(0, 3),
                       textcoords="offset points",
                       ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    return fig

def create_bit_width_analysis():
    """Plot metrics vs bit width for trend analysis"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy vs bit width
    ax1.plot(bit_widths, fpga_accuracy, 'o-', linewidth=2, markersize=8, label='FPGA Accuracy')
    ax1.set_xlabel('Bit Width', fontsize=12)
    ax1.set_ylabel('Accuracy (%)', fontsize=12)
    ax1.set_title('Accuracy Degradation vs Bit Width', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_xscale('log', base=2)
    ax1.set_xticks(bit_widths)
    ax1.set_xticklabels(bit_widths)
    
    # Throughput vs bit width
    ax2.plot(bit_widths, throughput, 's-', linewidth=2, markersize=8, color='orange', label='Throughput')
    ax2.set_xlabel('Bit Width', fontsize=12)
    ax2.set_ylabel('Throughput (nodes/sec)', fontsize=12)
    ax2.set_title('Throughput vs Bit Width', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.set_xscale('log', base=2)
    ax2.set_xticks(bit_widths)
    ax2.set_xticklabels(bit_widths)
    
    # Energy efficiency vs bit width
    energy_efficiency = [t/e for t, e in zip(throughput, energy_consumption)]
    ax3.plot(bit_widths, energy_efficiency, '^-', linewidth=2, markersize=8, color='green', label='Energy Efficiency')
    ax3.set_xlabel('Bit Width', fontsize=12)
    ax3.set_ylabel('Energy Efficiency (nodes/Joule)', fontsize=12)
    ax3.set_title('Energy Efficiency vs Bit Width', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.set_xscale('log', base=2)
    ax3.set_xticks(bit_widths)
    ax3.set_xticklabels(bit_widths)
    
    # Combined metrics (normalized)
    norm_accuracy = np.array(fpga_accuracy) / max(fpga_accuracy)
    norm_throughput = np.array(throughput) / max(throughput)
    norm_efficiency = np.array(energy_efficiency) / max(energy_efficiency)
    
    ax4.plot(bit_widths, norm_accuracy, 'o-', linewidth=2, markersize=6, label='Accuracy (norm)')
    ax4.plot(bit_widths, norm_throughput, 's-', linewidth=2, markersize=6, label='Throughput (norm)')
    ax4.plot(bit_widths, norm_efficiency, '^-', linewidth=2, markersize=6, label='Energy Eff. (norm)')
    ax4.set_xlabel('Bit Width', fontsize=12)
    ax4.set_ylabel('Normalized Score', fontsize=12)
    ax4.set_title('Normalized Metrics vs Bit Width', fontsize=12, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    ax4.set_xscale('log', base=2)
    ax4.set_xticks(bit_widths)
    ax4.set_xticklabels(bit_widths)
    ax4.legend()
    
    plt.tight_layout()
    return fig

def create_summary_table():
    """Create a summary table of all results"""
    data = {
        'Quantization': quantization_levels,
        'Training Acc (%)': training_accuracy,
        'FPGA Acc (%)': fpga_accuracy,
        'Precision': [f"{p:.3f}" for p in macro_precision],
        'Recall': [f"{r:.3f}" for r in macro_recall],
        'F1-Score': [f"{f:.3f}" for f in macro_f1],
        'Runtime (s)': [f"{r:.2f}" for r in runtime],
        'Throughput (nodes/s)': [f"{t:.0f}" for t in throughput],
        'Energy (J)': [f"{e:.2f}" for e in energy_consumption],
        'Energy Eff (nodes/J)': [f"{t/e:.0f}" for t, e in zip(throughput, energy_consumption)]
    }
    
    df = pd.DataFrame(data)
    
    fig, ax = plt.subplots(figsize=(16, 6))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    
    # Style the table
    for i in range(len(df.columns)):
        table[(0, i)].set_facecolor('#40466e')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    plt.title('GCN Quantization Results Summary', fontsize=16, fontweight='bold', pad=20)
    return fig

# Function to load and analyze classification report from npy file
def analyze_classification_report_from_npy(npy_file_path, true_labels_file=None):
    """
    Analyze classification results from numpy files
    
    Args:
        npy_file_path: Path to .npy file containing predictions
        true_labels_file: Path to true labels (if separate file)
    """
    predictions = np.load(npy_file_path)
    
    if true_labels_file:
        true_labels = np.load(true_labels_file)
    else:
        # You'll need to provide true labels somehow
        print("Please provide true labels for classification report generation")
        return None
    
    from sklearn.metrics import classification_report, precision_recall_fscore_support
    
    # Generate classification report
    report = classification_report(true_labels, predictions, output_dict=True)
    
    # Extract metrics
    precision = [report[str(i)]['precision'] for i in range(len(report)-3)]
    recall = [report[str(i)]['recall'] for i in range(len(report)-3)]
    f1 = [report[str(i)]['f1-score'] for i in range(len(report)-3)]
    
    return precision, recall, f1

if __name__ == "__main__":
    # Create all plots
    print("Generating GCN quantization analysis plots...")
    
    # Generate plots
    fig1 = create_accuracy_comparison_plot()
    fig1.savefig('gcn_accuracy_comparison.png', dpi=300, bbox_inches='tight')
    
    fig2 = create_performance_metrics_plot()
    fig2.savefig('gcn_performance_metrics.png', dpi=300, bbox_inches='tight')
    
    fig3 = create_classification_metrics_plot()
    fig3.savefig('gcn_classification_metrics.png', dpi=300, bbox_inches='tight')
    
    fig4 = create_bit_width_analysis()
    fig4.savefig('gcn_bit_width_analysis.png', dpi=300, bbox_inches='tight')
    
    fig5 = create_summary_table()
    fig5.savefig('gcn_results_summary.png', dpi=300, bbox_inches='tight')
    
    print("All plots saved successfully!")
    print("Files generated:")
    print("- gcn_accuracy_comparison.png")
    print("- gcn_performance_metrics.png") 
    print("- gcn_classification_metrics.png")
    print("- gcn_bit_width_analysis.png")
    print("- gcn_results_summary.png")
    
    # Show plots
    plt.show()

# Additional utility function for custom analysis
def custom_analysis_from_your_data(your_npy_predictions, your_true_labels):
    """
    Use this function with your actual .npy files
    Replace the estimated data above with real data from your files
    """
    # Load your data
    predictions = np.load(your_npy_predictions)
    labels = np.load(your_true_labels)
    
    # Generate metrics for each quantization level
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None)
    accuracy = accuracy_score(labels, predictions)
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Per-class Precision: {precision}")
    print(f"Per-class Recall: {recall}")
    print(f"Per-class F1: {f1}")
    
    return precision, recall, f1, accuracy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- Data ---
quant_levels = ['w16a16', 'w8a8', 'w4a4', 'w2a2']
fpga_accuracy = np.array([79.95, 79.87, 79.95, 74.85])
weighted_precision = np.array([0.8083, 0.8078, 0.8078, 0.7913]) * 100 # Convert to percentage for plotting alongside accuracy
weighted_recall = np.array([0.7995, 0.7987, 0.7995, 0.7485]) * 100    # Convert to percentage
weighted_f1_score = np.array([0.7998, 0.7991, 0.8003, 0.7490]) * 100 # Convert to percentage

pc_accuracy_32bit = 80.9  # For reference

runtimes = np.array([2.5408, 2.4667, 2.0962, 2.4624]) # seconds
throughput = np.array([1065.79, 1097.82, 1291.87, 1099.72]) # nodes/sec

# Estimated Energy Consumption (Joules)
# Power = 400mW = 0.4W
power_arm_core = 0.4 # Watts
estimated_energy = power_arm_core * runtimes # Joules

# --- Plotting Functions ---

def plot_grouped_bar_chart(ax, data_dict, title, ylabel, add_reference_line=None, ref_line_label=None, y_limit=None):
    """Plots a grouped bar chart."""
    n_groups = len(quant_levels)
    n_bars = len(data_dict)
    bar_width = 0.8 / n_bars # Adjust bar width based on number of metrics
    index = np.arange(n_groups)

    for i, (metric_name, values) in enumerate(data_dict.items()):
        bar_positions = index + i * bar_width - (bar_width * (n_bars -1) / 2) # Center the group
        bars = ax.bar(bar_positions, values, bar_width, label=metric_name)
        # Add text labels on top of bars
        for bar in bars:
            yval = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2.0, yval + 0.5, f'{yval:.2f}', ha='center', va='bottom', fontsize=8)


    ax.set_xlabel('Quantization Level (Weights & Activations)')
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_xticks(index + bar_width / n_bars - bar_width/2) # Adjust x-ticks to be centered for the group
    ax.set_xticklabels(quant_levels)
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.7)
    if y_limit:
        ax.set_ylim(y_limit)
    if add_reference_line is not None:
        ax.axhline(y=add_reference_line, color='r', linestyle='--', label=ref_line_label if ref_line_label else 'Reference')
        ax.legend() # Update legend to include reference line


def plot_single_bar_chart(ax, values, title, ylabel, add_reference_line=None, ref_line_label=None, y_limit=None):
    """Plots a single metric bar chart."""
    index = np.arange(len(quant_levels))
    bars = ax.bar(index, values, 0.6, label=ylabel) # Using 0.6 for bar width
    ax.set_xlabel('Quantization Level (Weights & Activations)')
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_xticks(index)
    ax.set_xticklabels(quant_levels)
    ax.grid(True, linestyle='--', alpha=0.7)

    # Add text labels on top of bars
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2.0, yval + (0.01 * (ax.get_ylim()[1] - ax.get_ylim()[0])), f'{yval:.2f}', ha='center', va='bottom', fontsize=9)


    if y_limit:
        ax.set_ylim(y_limit)
    if add_reference_line is not None:
        ax.axhline(y=add_reference_line, color='r', linestyle='--', label=ref_line_label if ref_line_label else 'Reference')
        ax.legend()


# --- Create Plots ---
plt.style.use('seaborn-v0_8-whitegrid') # Using a seaborn style for better aesthetics

# 1. Performance Metrics (Accuracy, Precision, Recall, F1-Score)
fig1, ax1 = plt.subplots(figsize=(12, 7))
performance_data = {
    'FPGA Accuracy (%)': fpga_accuracy,
    'Weighted Precision (%)': weighted_precision,
    'Weighted Recall (%)': weighted_recall,
    'Weighted F1-Score (%)': weighted_f1_score
}
plot_grouped_bar_chart(ax1, performance_data,
                       'GCN Performance Metrics on FPGA ARM Core',
                       'Score (%)',
                       add_reference_line=pc_accuracy_32bit,
                       ref_line_label=f'w32a32 PC Accuracy ({pc_accuracy_32bit}%)',
                       y_limit=[min(0, np.min(fpga_accuracy)-10), 100])
fig1.tight_layout()
plt.savefig('gcn_performance_metrics.png', dpi=300)
plt.show()


# 2. Throughput
fig2, ax2 = plt.subplots(figsize=(10, 6))
plot_single_bar_chart(ax2, throughput,
                      'GCN Throughput on FPGA ARM Core',
                      'Throughput (nodes/sec)')
fig2.tight_layout()
plt.savefig('gcn_throughput.png', dpi=300)
plt.show()

# 3. Estimated Energy Consumption
fig3, ax3 = plt.subplots(figsize=(10, 6))
plot_single_bar_chart(ax3, estimated_energy,
                      'Estimated Energy Consumption per Run on FPGA ARM Core',
                      'Energy (Joules)')
fig3.tight_layout()
plt.savefig('gcn_energy_consumption.png', dpi=300)
plt.show()

print("Graphs generated and saved as PNG files.")
print(f"Estimated Energy (Joules): {dict(zip(quant_levels, estimated_energy))}")